In [5]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
import warnings
from sklearn.exceptions import ConvergenceWarning

In [2]:
#Importing the dataset 
df=pd.read_csv("/home/umair/Desktop/Data Science and BI/data mining/occupancy_data/DataCleaned.csv")
test=pd.read_csv("/home/umair/Desktop/Data Science and BI/data mining/occupancy_data/DataCleanedTest.csv")
test2=pd.read_csv("/home/umair/Desktop/Data Science and BI/data mining/occupancy_data/DataCleanedTest2.csv")

In [3]:
#Drop the useless column
df.drop(['Unnamed: 0'], axis=1, inplace =True)
df.drop(['date'], axis=1, inplace =True)
df.drop(['DayName'], axis=1, inplace =True)

test.drop(['Unnamed: 0'], axis=1, inplace =True)
test.drop(['date'], axis=1, inplace =True)
test.drop(['DayName'], axis=1, inplace =True)

test2.drop(['Unnamed: 0'], axis=1, inplace =True)
test2.drop(['date'], axis=1, inplace =True)
test2.drop(['DayName'], axis=1, inplace =True)

In [4]:
#preparing the two datasets splitting the attributes with the class and selecting the right attributes
attributes = [col for col in df.columns if col != 'Occupancy']
X_train = df[attributes]
y_train = df['Occupancy']

attributes = [col for col in test.columns if col != 'Occupancy']
X_test = test[attributes]
y_test = test['Occupancy']

attributes = [col for col in test2.columns if col != 'Occupancy']
X_test2 = test2[attributes]
y_test2 = test2['Occupancy']

In [6]:
warnings.simplefilter("ignore")

In [9]:
#HyperTuning parameters with GridSearch crossing between training set, test set1
# and test set2. Searching for the best C
from sklearn.model_selection import GridSearchCV

features_combs_list = [
    ('IsWorkDay', 'Hour', 'IsWorkHour', 'Temperature', 'Humidity', 'Light', 'CO2')
]

hyper_params_space = [
    {
        'penalty': ['l1', 'l2'],
        'random_state': [0],
        'alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 5],
        'tol': 0.
    },
]

for features in features_combs_list:
    print(features)
    print('===================================')
    X = X_train
    X_t1 = X_test
    X_t2 = X_test2

    svc = GridSearchCV(Perceptron(), hyper_params_space,
                       scoring='accuracy')
    svc.fit(X, y_train)
    
    print('Best parameters set:')
    print(svc.best_params_)
    print()
    
    preds = [
        (svc.predict(X), y_train, 'Train'),
        (svc.predict(X_t1), y_test, 'Test1'),
        (svc.predict(X_t2), y_test2, 'Test2')
    ]
    
    for pred in preds:
        print('Accuracy %s' % accuracy_score(pred[1], pred[0]))
        print()
        print(pred[2] + ' Classification Report:')
        print()
        print(classification_report(pred[1], pred[0]))
        print()
        print(pred[2] + ' Confusion Matrix:')
        print(confusion_matrix(pred[1], pred[0]))
        print()

('IsWorkDay', 'Hour', 'IsWorkHour', 'Temperature', 'Humidity', 'Light', 'CO2')
Best parameters set:
{'alpha': 0.0001, 'penalty': 'l2', 'random_state': 0, 'tol': 0.1}

Accuracy 0.918580375782881

Train Classification Report:

              precision    recall  f1-score   support

           0       0.99      0.91      0.95      6414
           1       0.74      0.95      0.83      1729

    accuracy                           0.92      8143
   macro avg       0.86      0.93      0.89      8143
weighted avg       0.93      0.92      0.92      8143


Train Confusion Matrix:
[[5834  580]
 [  83 1646]]

Accuracy 0.8424015009380863

Test1 Classification Report:

              precision    recall  f1-score   support

           0       0.97      0.77      0.86      1693
           1       0.71      0.96      0.82       972

    accuracy                           0.84      2665
   macro avg       0.84      0.87      0.84      2665
weighted avg       0.88      0.84      0.85      2665


Test1 Co